###  <font color='green' size=5> HW #4: Air Quality - MongoDB Assignment  </font> 

**NoSQL** <br>
**ANLT 243** <br>
**September 14, 2018** <br>
<br>

<font color='green' size=4> By: Ali Taheri</font>

At first, we set reqired parameters for accessing "aq" collection in mongodb database of "nosql" on specified aws server.

In [25]:
from pymongo import MongoClient, ASCENDING, DESCENDING
import pprint

host = 'ec2-54-67-15-68.us-west-1.compute.amazonaws.com'
client = MongoClient(host, 9210)

db = client.nosql                       # db refers to nosql database
db_names = db.list_collection_names()   # accessing name of existing collectiones
print('\nList of collections in db')
pprint.pprint(db_names)


List of collections in db
['population', 'aq', 'population_Ali']


##  <font color='blue' size=4> 1. How many total records are there in this air quality collection ?  </font>

In following part, we access "aq" collection and will count number of documents by "aggregate" function.

In [26]:
aq=db.aq

counting_cursor=aq.aggregate([  
  { 
    "$group": {
      "_id" : None, 
      "count": {
        "$sum": 1
      }
    }
  }
])

print('\nNumber of documents in aq collecton is : ' + str(counting_cursor.next()['count']))



Number of documents in aq collecton is : 218635


##  <font color='blue' size=4> 2. What is the data type of the columns StateName, ReportYear, and Value ?  </font>

With accessing to a typical record, we extract type of mentioned columns.

In [27]:
typical_document = aq.find_one()

for column in ('StateName', 'ReportYear', 'Value'):
    print("\nType of " + column + " column is : "+ str(type(typical_document[column])))



Type of StateName column is : <class 'str'>

Type of ReportYear column is : <class 'int'>

Type of Value column is : <class 'float'>


##  <font color='blue' size=4> 3. What was the air quality metric value for MeasureId 83 in the Santa Barbara county in the state of California in 2013 ?  </font>

For finding air quality metric value for MeasureId 83 in the Santa Barbara county in the state of California in 2013, we find a record with following matching criteria, we use "find_one" function.

In [28]:
aq_desired_document = aq.find_one({ 'MeasureId': 83 , 'CountyName': 'Santa Barbara',
                                'StateName': 'California', 'ReportYear': 2013 })

print('\nAir quality metric value for MeasureId 83 in the Santa Barbara county in the state of California in 2013 was ' \
          + str(aq_desired_document['Value']) )


Air quality metric value for MeasureId 83 in the Santa Barbara county in the state of California in 2013 was 1.0


##  <font color='blue' size=4> 4. What is the MeasureName of MeasureId 85 ?  </font>

Same to above, we retrieve description for MeasureId of 85.

In [29]:
aq_desired_document = aq.find_one({ 'MeasureId': 85 })
print('\n MeasureName of MeasureId 85 is ' + str(aq_desired_document['MeasureName']))


 MeasureName of MeasureId 85 is Percent of days with PM2.5 levels over the National Ambient Air Quality Standard (NAAQS)


##  <font color='blue' size=4> 5. Which state and county have the highest recorded air quality value of MeasureId 85 in any year ?  </font>

For finding places with the most value of MeasureId of 85, first we extract years of measuring. Two different approaches have been mentioned below.

In [30]:
year_range = aq.aggregate([
    { "$match" : {'MeasureId': 85} } ,
    { 
    "$group": {
      "_id": None, 
      "from_year": {"$min": "$ReportYear"} ,
      "to_year":   {"$max": "$ReportYear"}
    }
 }
]).next()

from_year = year_range['from_year']
to_year = year_range['to_year']

#from_year = aq.find_one({'MeasureId': 85},sort=[("ReportYear", 1)])['ReportYear']
#to_year = aq.find_one({'MeasureId': 85},sort=[("ReportYear", -1)])['ReportYear']


print('\n Statistics for MeasureId of 85 are for years between '+ str(from_year) + ' and ' + str(to_year) + '.')

#from_year = aq.find_one({'MeasureId': 85},sort=[("ReportYear", 1)])['ReportYear']
#to_year = aq.find_one({'MeasureId': 85},sort=[("ReportYear", -1)])['ReportYear']


 Statistics for MeasureId of 85 are for years between 1999 and 2013.


Based on above info, we find the county with the highest value of MeasureId of 85 for each year in a for loop.

In [31]:
print('\n Following counties had the highest recorded air quality value of MeasureId 85 at each year:')

for year in range(from_year , to_year+1):
    max_record=aq.find_one({'MeasureId': 85 , "ReportYear": year},sort=[("Value", -1)])
    print("\n In " + str(year) + ", " + str(max_record['CountyName']) + ' from ' + str(max_record['StateName']) )



 Following counties had the highest recorded air quality value of MeasureId 85 at each year:

 In 1999, Riverside from California

 In 2000, Riverside from California

 In 2001, Riverside from California

 In 2002, San Bernardino from California

 In 2003, Riverside from California

 In 2004, Fairbanks North Star from Alaska

 In 2005, Allegheny from Pennsylvania

 In 2006, Allegheny from Pennsylvania

 In 2007, Kings from California

 In 2008, Kern from California

 In 2009, Kern from California

 In 2010, Silver Bow from Montana

 In 2011, Fresno from California

 In 2012, Kings from California

 In 2013, Kings from California


##  <font color='blue' size=4> 6. Print the average of air quality value and the unit for MeasureId 87, State - Florida, CountyName - Pinellas, for years between 2002 - 2008 ?  </font>

Now, for finding the average of air quality value and the unit for MeasureId of 87 in county of Pinellas from Florida,  for years between 2002 - 2008, we use "aggregate" function with following matching criteria.

In [32]:
aq_desired_result = aq.aggregate([
    { "$match" : {'MeasureId': 87, 'CountyName': 'Pinellas' , 'StateName': 'Florida' , \
                        'ReportYear': { "$lt": 2009 } , 'ReportYear': { "$gt": 2001 }}} ,
    { 
    "$group": {
      "_id": { "Unit": "$Unit"},
      "average_value": {
        "$avg": "$Value"
      }
    }
  }
    ]).next()

print('\nAverage of air quality value for MeasureId 87 in Pinellas of Florida between 2001 and 2008 is ' +
      str(round(aq_desired_result['average_value'] , 3)) +' ' + aq_desired_result["_id"]["Unit"])



Average of air quality value for MeasureId 87 in Pinellas of Florida between 2001 and 2008 is 8.653 µg/m³


##  <font color='blue' size=4> 7. Based on the data provided in the air quality collection, which state do you believe has better air quality- California or Texas ? Why? Call out any assumptions you make.  </font>

For comparison of two states of California and Texas, first we retrieve all years of records.

In [33]:
california_docs_years = sorted(aq.find({'StateName': 'California'}).distinct("ReportYear"))
texas_docs_years = sorted(aq.find({'StateName': 'Texas'}).distinct("ReportYear"))

print(california_docs_years)
print(texas_docs_years)


[1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013]
[1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013]


We see that we have records for both states between 1999 and 2013. <br>
For a decent comparison, we could consider last two years of 2012 and 2013. It's time to know which parameters have been measured at these days.

In [34]:
#All documents of California for 2012 and 2013

california_docs_2012_2013 = aq.find({'StateName': 'California','ReportYear': {"$gt" : 2011}})

#All documents of Texas for 2012 and 2013

texas_docs_2012_2013 = aq.find({'StateName': 'Texas' ,'ReportYear': {"$gt" : 2011}})

print(sorted(california_docs_2012_2013.distinct("MeasureId")))  #sorte list of measure ids of CA in 2012 and 2013
print(sorted(texas_docs_2012_2013.distinct("MeasureId")))       #sorte list of measure ids of TX in 2012 and 2013


[83, 84, 85, 86, 87]
[83, 84, 85, 86, 87]


So, we would calculate average of above parameters for two states in 2012 and 2013.

In [35]:
measure_ids = sorted(california_docs_2012_2013.distinct("MeasureId")) # 5 measure ids

california_aq_2012_2013=[]   #list of average of parameters for CA in 2012 and 2013

for Measure_ID in measure_ids:
    aq_desired_result = aq.aggregate([
    { "$match" : {'MeasureId': Measure_ID,'StateName': 'California' , 'ReportYear': {"$gt" : 2011} }} ,
    { 
    "$group": {
      "_id": None,
      "average_value": {
        "$avg": "$Value"
      }
    }
    }
    ]).next()
    
    california_aq_2012_2013.append(round(aq_desired_result["average_value"] ,2)) #appending averages to list
    

texas_aq_2012_2013=[]    #list of average of parameters for TX in 2012 and 2013

for Measure_ID in measure_ids:
    aq_desired_result = aq.aggregate([
    { "$match" : {'MeasureId': Measure_ID,'StateName': 'Texas' , 'ReportYear': {"$gt" : 2011} }} ,
    { 
    "$group": {
      "_id": None,
      "average_value": {
        "$avg": "$Value"
      }
    }
    }
    ]).next()
    
    texas_aq_2012_2013.append(round(aq_desired_result["average_value"] ,2))  #appending averages to list
    
#Subtraction of averages in CA and TX
print([round(x - y,2) for x, y in zip(california_aq_2012_2013, texas_aq_2012_2013)])


[9.89, 22567535.98, 2.8, 8368825.59, 0.06]


It is obvious that air pollution in California is higher than Texas and it's main reason could be a lot of wild fires in California. 